In [1]:
import pyspark
from pyspark.sql import Column, DataFrame, SparkSession, functions
from pyspark.sql.functions import *
from py4j.java_collections import MapConverter
import shutil
import random
import threading
conf = pyspark.SparkConf()
conf.setMaster("spark://spark:7077") 

conf.set("spark.hadoop.fs.s3a.endpoint", 'http://s3:9000') \
    .set("spark.hadoop.fs.s3a.access.key", 'minio') \
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
sc = pyspark.SparkContext(conf=conf)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.databricks#dbutils-api_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7ba30dcd-1be7-4714-ab94-977b748b8ee0;1.0
	confs: [default]
	found com.databricks#dbutils-api_2.12;0.0.5 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession(sc).builder.appName("streaming").getOrCreate()
from delta.tables import *

In [9]:
#raw data received from kafka is stored under S3 customers
OBJECTURL_TEST = 's3a://minio-sink-bucket/topics/customers'
schema=spark.read.format('json').load(OBJECTURL_TEST).schema
streamingRawDF=spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST)
stream=streamingRawDF.select(col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email")).withColumn('inserted', current_timestamp()).writeStream \
  .format("delta") \
  .outputMode("append") \
  .partitionBy(['last_name', 'first_name']) \
  .option("overwriteSchema", "true") \
  .trigger(once=True) \
  .options(ignoreDeletes=True) \
  .option("checkpointLocation", 's3a://minio-sink-bucket/delta/bronze/customers/checkpoints/').start('s3a://minio-sink-bucket/delta/bronze/customers/data/')
stream.awaitTermination()
#stream.stop()
deltaTable = DeltaTable.forPath(spark, 's3a://minio-sink-bucket/delta/bronze/customers/data')
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()

#############  Original Delta Table ###############


+----+----------+---------+--------------------+--------------------+
|  id|first_name|last_name|               email|            inserted|
+----+----------+---------+--------------------+--------------------+
|1001|      Jane|  changed|sally.thomas@acme...|2021-10-31 06:35:...|
|1001|     Sally|   Thomas|sally.thomas@acme...|2021-10-31 06:31:...|
|1005|      John|      Doe|john.doe@example.com|2021-10-31 06:35:...|
|1002|    George|   Bailey|  gbailey@foobar.com|2021-10-31 06:31:...|
|1004|      Anne|Kretchmar|  annek@noanswer.org|2021-10-31 06:31:...|
|1003|    Edward|   Walker|       ed@walker.com|2021-10-31 06:31:...|
+----+----------+---------+--------------------+--------------------+



In [4]:
df=spark.readStream.format("delta").load('s3a://minio-sink-bucket/delta/bronze/customers/data')
#df=df.withColumn('inserted', current_timestamp())

call this only on the 1st time in order to create the table.
need to find away to identify if the path is empty, similar to databricks dbuitls

In [5]:
df.writeStream \
  .format("delta") \
  .outputMode("append") \
  .trigger(once=True) \
  .option("checkpointLocation", 's3a://minio-sink-bucket/delta/bronze/customers_stream/checkpoints/') \
  .start('s3a://minio-sink-bucket/delta/bronze/customers_stream/data')

In [7]:
table_name = 'customerstream'
save_path='s3a://minio-sink-bucket/delta/bronze/customers_stream/data'
spark.sql("CREATE TABLE " + table_name + " USING DELTA LOCATION '" + save_path + "'")

DataFrame[]

In [8]:
deltaDf = DeltaTable.forName(spark, 'customerstream')

In [10]:
def mergetoDF(microDF, batchId):
    print(f"inside foreachBatch for batchId{batchId}. rows passed={microDF.count()}")
#    microDF.dropDuplicates("id")
    deltaDf.alias("t").merge(microDF.alias("s"), "s.id = t.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

In [11]:
df.writeStream \
  .format("delta") \
  .outputMode("append") \
  .foreachBatch(mergetoDF) \
  .trigger(once=True) \
  .option("checkpointLocation", 's3a://minio-sink-bucket/delta/bronze/customers_stream/checkpoints/') \
  .start()

21/10/31 06:36:18 WARN MicroBatchExecution: The read limit MaxFiles: 1000 for DeltaSource[s3a://minio-sink-bucket/delta/bronze/customers/data] is ignored when Trigger.Once() is used.


inside foreachBatch for batchId1. rows passed=2


the following is an example of stream data with upsert, in order to avoid duplication by id
the input is CSV file and the output is bronze delta with only rellevant id's
we might want to change the CSV to Avro, as its more efficient, but less readable
now, execute the insert and update from the README

In [12]:
deltaDf.toDF().show()

+----+----------+---------+--------------------+--------------------+
|  id|first_name|last_name|               email|            inserted|
+----+----------+---------+--------------------+--------------------+
|1001|      Jane|  changed|sally.thomas@acme...|2021-10-31 06:35:...|
|1004|      Anne|Kretchmar|  annek@noanswer.org|2021-10-31 06:31:...|
|1005|      John|      Doe|john.doe@example.com|2021-10-31 06:35:...|
|1002|    George|   Bailey|  gbailey@foobar.com|2021-10-31 06:31:...|
|1003|    Edward|   Walker|       ed@walker.com|2021-10-31 06:31:...|
+----+----------+---------+--------------------+--------------------+



In [13]:
deltaTable.history().show()

+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|       operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2021-10-31 06:35:46|  null|    null|STREAMING UPDATE|{outputMode -> Ap...|null|    null|     null|          0|          null|         true|{numRemovedFiles ...|        null|
|      0|2021-10-31 06:31:30|  null|    null|STREAMING UPDATE|{outputMode -> Ap...|null|    null|     null|       null|          null|         true|{numRemovedFiles ...|        null|
+-------+-------------------+------+--------+----------------+--------------------+--

example for updating according to input event

In [18]:
deltaTable.delete()

In [16]:
deltaTable.toDF().show()

+---+----------+---------+-----+--------+
| id|first_name|last_name|email|inserted|
+---+----------+---------+-----+--------+
+---+----------+---------+-----+--------+



In [20]:
deltaTable.vacuum()  

Deleted 0 files and directories in a total of 15 directories.


DataFrame[]